# Proof of Concept - ML with Homomorphic Encryption 

Basic idea is to try out very simple neural network in Torch to be converted into an equivalent network that processes homomorphic encrpyted data. 
In this example, we are using two libraries: TenSEAL by OpenMined and Concrete-ML by Zama. 

**Dataset used**: Adult income dataset 
https://www.kaggle.com/datasets/wenruliu/adult-income-dataset 

**Model architecture**:
- Linear (256, 15) // 256 neurons, 15 input features
- ReLU 
- Linear (2, 256)  // 2 output class

--- 
Version Log and Todo:

- Need to recheck on Tenseal context to perform multiple matrix multiplications
---

This Proof of Concept is largely influenced by 

https://github.com/OpenMined/TenSEAL/blob/main/tutorials/Tutorial%201%20-%20Training%20and%20Evaluation%20of%20Logistic%20Regression%20on%20Encrypted%20Data.ipynb#:~:text=Tutorial,-1%20%2D%20Training%20and

In [71]:
import random
from time import time

import matplotlib.pyplot as plt
import numpy
import numpy as np
import pandas as pd
import tenseal as ts
import torch
import torch.nn as nn
from concrete.ml.torch.compile import compile_torch_model
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Dataset preparation

In [49]:
df = pd.read_csv(".data/adult.csv")

In [50]:
class Dataset(torch.utils.data.Dataset):
      def __init__(self, df, train=True, scaler=None, one_hot_encoder=None):
          # Make a copy of the dataframe to avoid modifying the original
          cdf = df.copy()

          # Handle missing values
          cdf = cdf.replace('?', np.nan)

          # Fill missing values with the most common value for categorical features
          categorical_cols = ['workclass', 'occupation', 'native-country']
          for col in categorical_cols:
              cdf[col] = cdf[col].fillna(cdf[col].mode()[0])

          # Process target column first (before one-hot encoding)
          if train:
              cdf['income'] = cdf['income'].map({'>50K': 1, '<=50K': 0})
              target = cdf['income'].values
              cdf.drop('income', axis=1, inplace=True)
          else:
              # For test set
              if 'income' in cdf.columns:
                  cdf['income'] = cdf['income'].map({'>50K': 1, '<=50K': 0})
                  target = cdf['income'].values
                  cdf.drop('income', axis=1, inplace=True)
              else:
                  target = None

          # Extract numeric and categorical columns
          numeric_cols = ['age', 'fnlwgt', 'educational-num', 'capital-gain', 'capital-loss',
  'hours-per-week']
          cat_cols = [col for col in cdf.columns if col not in numeric_cols]

          # Normalize numeric columns
          if train:
              # If training data, fit a new scaler
              self.scaler = StandardScaler()
              cdf[numeric_cols] = self.scaler.fit_transform(cdf[numeric_cols])
          else:
              # If test data, use the provided scaler
              if scaler is not None:
                  cdf[numeric_cols] = scaler.transform(cdf[numeric_cols])

          # One-hot encode categorical features
          if train:
              # Save original categorical data for reference
              self.categorical_data = cdf[cat_cols].copy()
              # Apply one-hot encoding
              cdf_encoded = pd.get_dummies(cdf, columns=cat_cols)
              # Save column names for future reference
              self.encoded_columns = cdf_encoded.columns
              processed_data = cdf_encoded
          else:
              # For test data, ensure we have the same columns as training
              if one_hot_encoder is not None:
                  # One-hot encode the categorical columns
                  test_encoded = pd.get_dummies(cdf, columns=cat_cols)

                  # Add missing columns that are in training but not in test
                  for col in one_hot_encoder:
                      if col not in test_encoded.columns:
                          test_encoded[col] = 0

                  # Ensure columns are in the same order as training
                  processed_data = test_encoded[one_hot_encoder]

          # Convert to numpy array
          self.data = processed_data.values.astype('float32')
          self.target = target
          self.nfeatures = self.data.shape[1]
          self.train = train

      def __len__(self):
          return len(self.data)

      def __getitem__(self, idx):
          if self.target is not None:
              return self.data[idx], self.target[idx]
          else:
              return self.data[idx], np.nan

In [ ]:
TEST_BATCH_SIZE  = 64
TRAIN_BATCH_SIZE = 512

In [ ]:
# Split the dataset into training and validation sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Create training dataset first
train_dataset = Dataset(train_df, train=True)

# Create test dataset using the same scaler and one-hot encoding as training
test_dataset = Dataset(test_df, train=False,
                    scaler=train_dataset.scaler,
                    one_hot_encoder=train_dataset.encoded_columns)

# Create data loaders
dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=TEST_BATCH_SIZE, shuffle=False)

# Verify shapes match (features should be same dimension)
print(f"Training data shape: {train_dataset.data.shape}")
print(f"Test data shape: {test_dataset.data.shape}")
print(f"Number of features: {train_dataset.nfeatures}")

Training data shape: (39073, 105)
Test data shape: (9769, 105)
Number of features: 105


In [77]:
next(iter(dataloader))

[tensor([[-0.3368,  0.2640,  1.1367,  ...,  1.0000,  0.0000,  0.0000],
         [-0.4825,  0.0210,  1.5256,  ...,  1.0000,  0.0000,  0.0000],
         [-1.0655,  2.9288,  1.1367,  ...,  1.0000,  0.0000,  0.0000],
         ...,
         [-0.8469, -0.3966, -0.0298,  ...,  1.0000,  0.0000,  0.0000],
         [-0.9926,  0.0554, -0.4187,  ...,  1.0000,  0.0000,  0.0000],
         [-1.5027,  1.6702, -0.4187,  ...,  1.0000,  0.0000,  0.0000]]),
 tensor([1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1,
         0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0,
         0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1,
         0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
         1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
         1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0,
         0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 

In [78]:
next(iter(dataloader))[0].shape

torch.Size([512, 105])

In [79]:
next(iter(testloader))[0].shape

torch.Size([64, 105])

# Model definition

In [80]:
class SimpleClassifier(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Linear(64, 2),
        )
        
    def forward(self, x):
        return self.model(x)

In [81]:
# no need for mps / cuda as its a very simple network
device = torch.device("cpu")

In [82]:
classifier = SimpleClassifier(train_dataset.nfeatures).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classifier.parameters(), lr=0.0001)

In [83]:
for epoch in range(10):
    running_correct = 0
    running_total = 0
    
    classifier.train()
    
    for i, (data, target) in enumerate((dataloader)):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        
        output = classifier(data)
        
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        pred = output.argmax(dim=1)
        running_correct += (pred == target).sum().item()
        running_total += target.size(0)
    
    print(f"epoch: {epoch+1}")
    print(f"  loss: {loss.item()}")
    print(f"  accuracy: {running_correct / running_total}")

epoch: 1
  loss: 0.6763036251068115
  accuracy: 0.3719448212320528
epoch: 2
  loss: 0.625588059425354
  accuracy: 0.7778773065799913
epoch: 3
  loss: 0.5257165431976318
  accuracy: 0.7813579709774012
epoch: 4
  loss: 0.4393138885498047
  accuracy: 0.7922094540987382
epoch: 5
  loss: 0.4221590757369995
  accuracy: 0.8093824380006654
epoch: 6
  loss: 0.366595596075058
  accuracy: 0.8234330611931513
epoch: 7
  loss: 0.3580872118473053
  accuracy: 0.8302664243851253
epoch: 8
  loss: 0.39628124237060547
  accuracy: 0.8355642003429478
epoch: 9
  loss: 0.3669053912162781
  accuracy: 0.8376372431090523
epoch: 10
  loss: 0.3955754339694977
  accuracy: 0.8398126583574336


---

# Concrete ML

## Understanding Homomorphic Encryption with Concrete ML

Homomorphic Encryption (HE) allows computation on encrypted data without decrypting it first. This preserves privacy while still enabling machine learning inference.

In this section, we'll explore how to:
1. Compile our PyTorch model with Concrete ML
2. Perform inference in simulation mode to verify accuracy
3. Execute inference in FHE mode to measure performance

### Step 1: Model Compilation

Here we compile our trained PyTorch model to be compatible with FHE operations. This process involves:
- Quantizing the model (reducing precision to work with integers)
- Creating an FHE-compatible circuit

In [84]:
# Create a representative input for compilation
torch_input = torch.randn(32, 105)

# Compile the PyTorch model
# - n_bits=6: Set quantization bits for weights and activations
# - rounding_threshold_bits: Controls precision during computation
quantized_module = compile_torch_model(
    classifier.to('cpu'),  # Our trained model
    torch_input,           # Sample input for quantization and compilation
    n_bits=6,              # Bit precision for quantization
    rounding_threshold_bits={"n_bits": 6, "method": "approximate"}
)

quantized_module

### Step 2: Simulation Mode Testing

In this step, we'll run inference in simulation mode (`fhe="simulate"`) to:
- Verify the model works with encrypted data
- Measure the accuracy impact of quantization
- Test on the first batch from our test dataset

This simulates FHE operations without actually encrypting data, which is much faster.

In [85]:
%%time 
# Get a single test example
data, target = next(iter(testloader))
data = data[0].unsqueeze(0)
target = target[0]

# Original PyTorch model prediction (baseline)
with torch.no_grad():
    original_output = classifier(data)
    original_pred = original_output.argmax().item()

print(f"Original model prediction: {original_pred}")
print(f"Actual target: {target.item()}")

# Quantize the input
x_test_q = quantized_module.quantize_input(data.numpy())

# Run inference in simulate mode (simulates FHE behavior without encryption)
y_pred_simulate = quantized_module.quantized_forward(x_test_q, fhe="execute")

# Dequantize the output to get back to original scale
y_pred_dequantized = quantized_module.dequantize_output(y_pred_simulate)
simulate_pred = np.argmax(y_pred_dequantized)

print(f"\nSimulated FHE prediction: {simulate_pred}")
print(f"Prediction matches original: {original_pred == simulate_pred}")
print(f"Both match actual target: {original_pred == target.item() and simulate_pred == target.item()}")

# Show quantized and dequantized values to observe any precision loss
print(f"\nOriginal output: {original_output.detach().numpy()}")
print(f"Dequantized output: {y_pred_dequantized}")
print(f"Quantization impact (difference): {original_output.detach().numpy() - y_pred_dequantized}")

Original model prediction: 0
Actual target: 0

Simulated FHE prediction: 0
Prediction matches original: True
Both match actual target: True

Original output: [[ 1.5171748 -1.2382808]]
Dequantized output: [[ 1.45416066 -1.23776891]]
Quantization impact (difference): [[ 0.06301418 -0.00051186]]
CPU times: user 1min 36s, sys: 2.33 s, total: 1min 39s
Wall time: 16.6 s


In [86]:
data[0].unsqueeze(0)

tensor([[ 1.2663, -1.4850, -0.4187, -0.1445, -0.2197, -0.0344,  0.0000,  0.0000,
          0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  1.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  

In [91]:
%%time
# Get a single test sample (using only the first item in the batch)
rand_id = np.random.randint(1, TEST_BATCH_SIZE)

data, target = next(iter(testloader))
data_single = data[rand_id].unsqueeze(0)  # Take just the first sample and keep batch dimension
target_single = target[rand_id].item()    # Get the actual target value

# Quantize the input
x_test_q = quantized_module.quantize_input(data_single.numpy())

# Run inference in simulation mode
print("Running in FHE simulate mode...")
y_pred_simulate = quantized_module.quantized_forward(x_test_q, fhe="simulate")

# Dequantize the output
y_pred_dequantized = quantized_module.dequantize_output(y_pred_simulate)
simulate_pred = np.argmax(y_pred_dequantized)

print(f"\nFHE simulation prediction: {simulate_pred}")
print(f"Actual target: {target_single}")
print(f"Prediction matches target: {simulate_pred == target_single}")

# Show the output
print(f"\nFHE output (dequantized): {y_pred_dequantized}")

# Compare with original model prediction
with torch.no_grad():
    original_output = classifier(data_single)
    original_pred = original_output.argmax().item()
    
print(f"Original model prediction: {original_pred}")
print(f"Match between FHE simulation and original: {original_pred == simulate_pred}")

NameError: name 'TEST_BATCH_SIZE' is not defined

In [89]:
%%time

# Get a test batch
data, target = next(iter(testloader))

# Run prediction with original PyTorch model
with torch.no_grad():
    original_output = classifier(data)  # Using the original PyTorch model
    original_pred = original_output.argmax(dim=1)
    original_accuracy = accuracy_score(target.numpy(), original_pred.numpy())

# Quantize the input for FHE compatibility
x_test_q = quantized_module.quantize_input(data.numpy())

# Run the forward pass in simulated FHE mode
y_pred_fhe = quantized_module.quantized_forward(x_test_q, fhe="simulate")

# Dequantize the output back to floating point
y_pred_fhe_dequantized = quantized_module.dequantize_output(y_pred_fhe)

# Get class predictions
fhe_pred = torch.tensor(np.argmax(y_pred_fhe_dequantized, axis=1))

# Calculate batch accuracy
accuracy = accuracy_score(target.numpy(), fhe_pred.numpy())

# Display results
print(f"\nFHE Execution Results:")
print(f"Prediction: {fhe_pred}")
print(f"Actual target: {target}")
print(f"Batch accuracy: {accuracy:.4f} ({torch.sum(fhe_pred == target).item()}/{len(target)} correct)")

# Show the output (dequantized values)
# print(f"\nFHE output (dequantized):")
# print(f"{y_pred_fhe_dequantized}")

# Compare with the original model
print(f"\nOriginal Model Results:")
print(f"Original model predictions: {original_pred}")
print(f"Original model accuracy: {original_accuracy:.4f} ({torch.sum(original_pred == target).item()}/{len(target)} correct)")
print(f"Agreement between FHE and original model: {torch.sum(fhe_pred == original_pred).item()}/{len(fhe_pred)} samples")

# Results using the classifier (if different from model)
print(f"\nClassifier Model Results:")
with torch.no_grad():
    classifier_output = classifier(data)
    classifier_pred = classifier_output.argmax(dim=1)
    classifier_accuracy = accuracy_score(target.numpy(), classifier_pred.numpy())
    
print(f"Classifier predictions: {classifier_pred}")
print(f"Classifier accuracy: {classifier_accuracy:.4f} ({torch.sum(classifier_pred == target).item()}/{len(target)} correct)")
print(f"Agreement between FHE and classifier: {torch.sum(fhe_pred == classifier_pred).item()}/{len(fhe_pred)} samples")


FHE Execution Results:
Prediction: tensor([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
        0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0])
Actual target: tensor([0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0])
Batch accuracy: 0.8750 (56/64 correct)

Original Model Results:
Original model predictions: tensor([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
        0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0])
Original model accuracy: 0.8750 (56/64 correct)
Agreement between FHE and original model: 64/64 samples

Classifier Model Results:
Classifier predictions: tensor([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0

### Step 3: FHE Execution Mode Performance

Now we'll run the model in full FHE execution mode (`fhe="execute"`). This performs actual homomorphic encryption operations:
1. Encrypts the input
2. Performs computation on encrypted data
3. Decrypts the result

We'll use `%%time` magic to measure performance, as FHE operations are typically much slower than plaintext operations.

In [ ]:
%%time
# Get a single test sample (using only the first item in the batch)
data, target = next(iter(testloader))
data_single = data[0].unsqueeze(0)  # Take just the first sample and keep batch dimension
target_single = target[0].item()    # Get the actual target value

# Quantize the input
x_test_q = quantized_module.quantize_input(data_single.numpy())

# Run inference in full FHE execution mode
print("Running in FHE execute mode (this may take some time)...")
y_pred_fhe = quantized_module.quantized_forward(x_test_q, fhe="execute")

# Dequantize the output
y_pred_fhe_dequantized = quantized_module.dequantize_output(y_pred_fhe)
fhe_pred = np.argmax(y_pred_fhe_dequantized)

print(f"\nFHE execution prediction: {fhe_pred}")
print(f"Actual target: {target_single}")
print(f"Prediction matches target: {fhe_pred == target_single}")

# Show the output
print(f"\nFHE output (dequantized): {y_pred_fhe_dequantized}")

# Compare with simulation results
print(f"\nComparing with previous results:")
with torch.no_grad():
    original_output = classifier(data_single)
    original_pred = original_output.argmax().item()
    
print(f"Original model prediction: {original_pred}")
print(f"FHE execution matches original model: {fhe_pred == original_pred}")

# If y_pred_simulate was stored from the previous cell
try:
    y_pred_simulate_dequantized = quantized_module.dequantize_output(y_pred_simulate)
    simulate_pred = np.argmax(y_pred_simulate_dequantized)
    print(f"FHE execution matches FHE simulation: {fhe_pred == simulate_pred}")
except NameError:
    print("No simulate results available for comparison")

### Step 3.1 Batch Processing in Simulation Mode



In [21]:
# Detailed breakdown of the FHE pipeline steps

# Get a sample input
data, target = next(iter(testloader))

# Step 1: Quantize the input (convert floats to integers)
# This is necessary since FHE works with integers
q_input = quantized_module.quantize_input(data.numpy())
print(f"1. Original input shape: {data.shape}")
print(f"   Quantized input: {q_input}")
print(f"   Quantized input shape: {q_input.shape}")
print()

# Step 2: Encrypt the quantized input
# The input is now encrypted and cannot be read without the decryption key
q_input_enc = quantized_module.fhe_circuit.encrypt(q_input)
print(f"2. Input encrypted: {type(q_input_enc)}")
print(f"   First few bytes of encrypted data: {q_input_enc.serialize().hex()[:100]}...")
print()

# Step 3: Execute the model on encrypted data
# The model performs calculations without ever decrypting the data
print("3. Running model on encrypted data...")
q_y_enc = quantized_module.fhe_circuit.run(q_input_enc)
print(f"   Model execution complete")
print(f"   Output is still encrypted: {type(q_y_enc)}")
print()

# Step 4: Decrypt the result (still quantized)
# Now we decrypt to get back the quantized prediction
q_y = quantized_module.fhe_circuit.decrypt(q_y_enc)
print(f"4. Decrypted result (still quantized): {q_y}")
print()

# Step 5: Dequantize the result (convert back to floating point)
# Convert from integer back to float for final prediction
y_final = quantized_module.dequantize_output(q_y)
print(f"5. Final dequantized output: {y_final}")
print(f"   Predicted class: {np.argmax(y_final)}")
print(f"   Actual class: {target.item()}")
print()

# Compare with direct model prediction (without FHE)
with torch.no_grad():
    direct_output = classifier(data).numpy()
print(f"6. Direct model output (without FHE): {direct_output}")
print(f"   Direct predicted class: {np.argmax(direct_output)}")
print(f"   Accuracy preserved: {np.argmax(direct_output) == np.argmax(y_final)}")

1. Original input shape: torch.Size([1, 105])
   Quantized input: [[ 10 -16  -6  -3  -4  -2  -2  -2  -2   7  -2  -2  -2  -2  -2  -2  -2  -2
   -2  -2  -2  -2  -2  -2  -2   7  -2  -2  -2  -2   7  -2  -2  -2  -2  -2
   -2  -2  -2  -2  -2  -2  -2  -2   7  -2  -2  -2  -2  -2  -2  -2  -2  -2
   -2   7  -2  -2  -2  -2  -2   7   7  -2  -2  -2  -2  -2  -2  -2  -2  -2
   -2  -2  -2  -2  -2  -2  -2  -2  -2  -2  -2  -2  -2  -2  -2  -2  -2  -2
   -2  -2  -2  -2  -2  -2  -2  -2  -2  -2  -2  -2   7  -2  -2]]
   Quantized input shape: (1, 105)

2. Input encrypted: <class 'concrete.fhe.compilation.value.Value'>
   First few bytes of encrypted data: 0100000057010000030000000000000000000000000003000800000000000100f80400000100010008050000010001000100...

3. Running model on encrypted data...
   Model execution complete
   Output is still encrypted: <class 'concrete.fhe.compilation.value.Value'>

4. Decrypted result (still quantized): [[ 5228 -5273]]

5. Final dequantized output: [[ 1.95626007 -2.17794744

# Tenseal

In [93]:
classifier.model[0].weight.data.shape

torch.Size([64, 105])

In [94]:
# Create Tenseal Context 

# parameters
poly_mod_degree = 16384
coeff_mod_bit_sizes = [60, 40, 40, 60]
# create TenSEALContext
ctx_eval = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)
# scale of ciphertext to use
ctx_eval.global_scale = 2 ** 40
# this key is needed for doing dot-product operations
ctx_eval.generate_galois_keys()


# # parameters
# poly_mod_degree = 16384  # Increased for more precision
# coeff_mod_bit_sizes = [50, 30, 30, 30, 50]  # More conservative chain
# scale = 2 ** 30  # Matched with middle layers

# # create TenSEALContext
# ctx_eval = ts.context(
#     scheme=ts.SCHEME_TYPE.CKKS,
#     poly_modulus_degree=poly_mod_degree,
#     coeff_mod_bit_sizes=coeff_mod_bit_sizes
# )
# ctx_eval.global_scale = scale
# ctx_eval.generate_galois_keys()

# ctx_eval = ts.context(ts.SCHEME_TYPE.CKKS, poly_modulus_degree = 16384, coeff_mod_bit_sizes = [60, 60, 60, 60, 60, 60])
# ctx_eval.generate_galois_keys()
# ctx_eval.global_scale = 2**25

In [95]:
# Testing 
a = torch.rand((1, 5))
b = torch.ones((5, 2))

enc_a = ts.ckks_tensor(ctx_eval, a)
enc_b = ts.ckks_tensor(ctx_eval, b)

(a @ b)

tensor([[2.5149, 2.5149]])

In [96]:
torch.tensor((enc_a @ enc_b).decrypt().tolist())

tensor([[2.5149, 2.5149]])

In [97]:
print(type(enc_a))
print(type(b))
torch.tensor((enc_a @ b).decrypt().tolist())

<class 'tenseal.tensors.ckkstensor.CKKSTensor'>
<class 'torch.Tensor'>


tensor([[2.5149, 2.5149]])

In [98]:
x, _ = next(iter(testloader))
enc_x = ts.ckks_tensor(ctx_eval, x.tolist())

print(x)
print(type(enc_x))
print(type(classifier.model[0].weight.data))
enc_x @ (classifier.model[0].weight.data.cpu().T)

: 

In [ ]:
enc_x.scale

: 

: 

Simulate a forward pass - 

In [ ]:
w_ = classifier.model[0].weight.data.cpu().T
w_.to(torch.float8_e4m3fn)
xl = enc_x @ (classifier.model[0].weight.data.cpu().T)

: 

: 

In [ ]:
# Stuck at this point, scale out of bounds

w2_ = classifier.model[2].weight.data.cpu().T
w2_.to(torch.float8_e5m2)

xl @ (w2_)

: 

: 

In [ ]:
# Code in this cell and after did not consider RELU yet, so need to adapt (hardcoded 2 layers)
class EncryptedClassifier:
    
    def __init__(self, unencrypted_model):
        
        # assuming same architecture
        self.w1 = unencrypted_model.model[0].weight.data.cpu() # [256, 15]
        self.b1 = unencrypted_model.model[0].bias.data.cpu()  # [256]
        
        self.w2 = unencrypted_model.model[2].weight.data.cpu()  # [2, 256]
        self.b2 = unencrypted_model.model[2].bias.data.cpu()  # [2]
        

    # def forward(self, enc_x):
    #     # assume encrypted input
        
    #     # enc_x would be in the shape of [1, 15]
    #     # while w1 is in shape [256, 15] 
    #     results = [ ]
    #     for row in self.w1.T:
    #         part_result = enc_x @ row  # Dot product instead of full matrix mult
    #         results.append(part_result) # a little tricky here since the individual items are in form ckks_tensor, how to stack them (?)
        
    #     # Combine results
    #     x = torch.stack(results)
    #     x = enc_x @ self.w1.T 
    #     x = x @ self.w2.T 
        
    #     return x
 
 
    ## apparently, the context need to be adjusted so we can do matmul twice.
    ## it keeps making scale out of bounds error. To be researched
    def forward(self, enc_x):
        # assume encrypted input
        
        # enc_x would be in the shape of [1, 15]
        # while w1 is in shape [256, 15] 
        
        x = enc_x @ self.w1.T 
        # need relu, but not sure how
        x = x @ self.w2.T 
        
        return x        
        
        
        # enc_z = []
        
        # # calculate dot product of each neuron
        # for i in range(self.w1.shape[0]):               # 256 times
        #     # print(f"forward 1 - {i}")
        #     z = enc_x.dot(self.w1[i, :])    # dot product input x neuron weights
        #     z += self.b1[i]                 # add bias value 
        #     enc_z.append(z)         # add the result
            
        # # enc_z should be an array of [256, 1]
        # print(type(enc_z[0]))
        # print(type(enc_z))
        
        # # redo for second layer
        # enc_z2 = []
        # # calculate dot product of each neuron
        # for i in range(self.w2.shape[0]):   # 2 times
        #     print(f"forward 2 - {i}")
        #     z = enc_z.dot(self.w2[i, :])    # dot product input x neuron weights
        #     z += self.b2[i]                 # add bias value 
        #     enc_z2.append(enc_z2)         # add the result
                  
        # # enc_z should be an array of [2, 1]
        # return enc_z

    
    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)
        

    
    def encrypt(self, context):
        self.w1 = ts.ckks_vector(context, self.w1)
        self.b1 = ts.ckks_vector(context, self.b1)
        
        
        self.w2 = ts.ckks_vector(context, self.w2)
        self.b2 = ts.ckks_vector(context, self.b2)
        
    def decrypt(self, context):
        self.w1 = self.w1.decrypt()
        self.b1 = self.b1.decrypt()
        
        self.w2 = self.w2.decrypt()
        self.b2 = self.b2.decrypt()
        

encCls = EncryptedClassifier(classifier)

: 

: 

In [ ]:
data, target = next(iter(testloader))

: 

: 

In [ ]:
for i, (data, target) in enumerate((testloader)):
    print(f"Instance {i}")
    
    x = data[0]
    print(f"\t input tensor: {x}")
    
    enc_x = ts.ckks_tensor(ctx_eval, [x])
    print(f"\t ckks enc tensor: {enc_x}")
    
    # encrypted evaluation
    enc_out = encCls(enc_x)
    print(f"\t forward output tensor: {enc_out}")
    
    # plain comparison
    out = enc_out.decrypt()
    print(f"\t decrypted output: {torch.tensor(out.tolist())}")
    
    if i == 5:
        break

: 

: 

: 

: 